# Problème TwoEq6

In [1]:
using ForwardDiff, JuMP, Ipopt

In [2]:
include("../src/enlsip_functions_castor2.jl")

enlsip

In [13]:
n = 2
m = 2
nb_eq = 0
nb_constraints = 2

2

In [4]:
# Résidus

function r(x::Vector)
    return [x[1]/(1-x[1]) - 5*log(0.4*(1-x[1])/x[2]) + 4.45977;
        x[2] - 0.4 + x[1]/2]
end
resTwoeq6 = ResidualsEval(0)

function (resTwoeq6::ResidualsEval)(x::Vector, rx::Vector, J::Matrix)
    
    if resTwoeq6 == 1
        rx[:] = [x[1]/(1-x[1]) - 5*log(0.4*(1-x[1])/x[2]) + 4.45977;
        x[2] - 0.4 + x[1]/2]
        
    elseif resTwoeq6 == 2
        J[:] = ForwardDiff.jacobian(r,x)
    end
    return
end

In [11]:
# Contraintes 

function c(x::Vector)
    return [x[1] - 0.0001; x[2] - 0.0001]
end

consTwoeq6 = ConstraintsEval(0)

function(consTwoeq6::ConstraintsEval)(x::Vector, cx::Vector, A::Matrix)
    
    if consTwoeq6.ctrl == 1
        cx[:] =  [x[1] - 0.0001; x[2] - 0.0001]
    elseif consTwoeq6.ctrl == 2
        A[:] = ForwardDiff.jacobian(c,x)
    end
    return
end

## Résolution avec ENLSIP-Julia

In [16]:
x0 = [.6,.1]  # [.9, .5] # 

e = eps(Float64)
se = sqrt(e)
enlsipTwoeq6 = enlsip(x0,resTwoeq6,consTwoeq6,n,m,nb_eq,nb_constraints,ε_abs = e, ε_rel = se, ε_x = se, ε_c = se)


****************************************
*                                      *
*          ENLSIP-JULIA-0.4.0          *
*                                      *
****************************************

Starting point : [0.6, 0.1]

Starting point :
    6.000000e-01   1.000000e-01 

Number of equality constraints   : 0
Number of inequality constraints : 2
Constraints internal scaling     : false


Iteration steps information

iter     objective    cx_sum   reduction     ||p||   dimA  dimJ2     α     conv. speed   max weight   working set
   0  0.000000e+00  0.00e+00   0.00e+00   0.000e+00   0     0   1.00e+00    0.00e+00     1.00e-01     -


LoadError: IOError: unlink("output.txt"): resource busy or locked (EBUSY)

## Résolution avec Ipopt 

In [20]:
x_low = [.0001, .0001]
x_up = [0.9999, Inf]

# x0 = [.6,.1]   
x0 = [.9, .5] 

model = Model(Ipopt.Optimizer)

@variable(model, x_low[i] <= x[i=1:n] <= x_up[i], start = x0[i])

@NLobjective(model, Min, (x[1]/(1-x[1]) - 5*log(0.4*(1-x[1])/x[2]) + 4.45977)^2 + (x[2] - 0.4 + x[1]/2)^2)

optimize!(model)

value.(model[:x])

This is Ipopt version 3.14.4, running with linear solver MUMPS 5.4.1.

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:        3

Total number of variables............................:        2
                     variables with only lower bounds:        1
                variables with lower and upper bounds:        1
                     variables with only upper bounds:        0
Total number of equality constraints.................:        0
Total number of inequality constraints...............:        0
        inequality constraints with only lower bounds:        0
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  6.8090780e+02 0.00e+00 1.00e+02  -1.0 0.00e+00    -  0.00e+00 0.00e+00  

2-element Vector{Float64}:
 0.7573874485755303
 0.021303286329152085